# RWA Data Quality Framework

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
import getpass
from tqdm.notebook import trange, tqdm_notebook

## User Input <a class="anchor" id="section_2"></a>

In [2]:
#Input variables
rd = "20220331"
#ws = "0002"
ws = "0210" #NRT_PRELIM
input_file = "input/template_RAY_QualityChecks_v3.xlsx"

#Define working directory
os.chdir(os.path.dirname(os.path.realpath("__file__")))

In [3]:
#Setting up RAY connection:
connection = create_engine(str("oracle://:@PN"))
connection.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                     reporting_date = rd, partition = ws))

## Import SQL & Query

In [4]:
#Import input excel
df_qc_list = pd.read_excel(input_file, sheet_name = "QC_list")

In [5]:
#create ACTIVE list
df_qc_list_active = df_qc_list[df_qc_list["STATUS"] == "ACTIVE"]

#create ACTIVE list without internal comments for NWU distribution
df_qc_list_active_nwu = df_qc_list_active.drop("Comments/updates", axis=1)

In [6]:
#Create list of SQL-s to be run
sql_list = df_qc_list[(pd.isnull(df_qc_list["ID"]) != True) & (df_qc_list["STATUS"] == "ACTIVE") &
                      (pd.isnull(df_qc_list["SQL"]) != True)][["ID","SQL"]]

In [7]:
#Query data
d = {} #create empty dictionary to store multiple dataframes

for i in tqdm_notebook(sql_list["ID"]): #looping through the ID-s
    sql = sql_list[sql_list["ID"] == i]["SQL"].values[0] #retreive SQL string string corresponding to ID
    d[i] = pd.read_sql_query(sql,connection) #query data

  0%|          | 0/4 [00:00<?, ?it/s]

## Export to EXCEL

### Overview - export all units to a single excel

In [8]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('output/RWA_DQ_checks_' + rd + '_(Timestamp_{}).xlsx'
                        .format(pd.datetime.today().strftime('%Y%m%d_%Hh%M')), engine='xlsxwriter')

#write overview of checks to first sheet
df_qc_list_active.to_excel(writer, sheet_name="QC_list")

# Write each dataframe to a different worksheet.
for i in sql_list["ID"]:
    d[i].to_excel(writer, sheet_name=i)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  This is separate from the ipykernel package so we can avoid doing imports until


### NWU - export units into separate excel files

In [ ]:
#Get list of units that have data returned in at least one quality check
unit_list_data = [] #empty list

for i in sql_list["ID"]: #get all units across all quality checks
    unit_list_data.extend(d[i]["unit_code"].unique())
    
unit_list_data = list(dict.fromkeys(unit_list_data)) #remove duplicates from final list

In [ ]:
#define master list of unit codes to split data for
unit_list_master = ["RBI"
    #banks
    ,"AVAL","CENTRO","KATHREIN","RBAL","RBBG","RBBH","RBBY","RBCN","RBCZ","RBHR","RBHU","RBIPL"
    ,"RBKO","RBRO","RBRS","RBRU","RBSG","RBSPK","RBSPKHR","RBSPKRO","TBSK"     
    #leasing
    ,"RLUA","RLAL","RLBG","RLBH","RLBY","RLCZ","RLHR","RLKO","RLRO","RLRS","RLRU,","TLSK","ILSK","RPL","RSTS"
    #special
    ,"RLAT"
     ]

In [ ]:
#limit export to intersection of unitst of returned data and the fixed list
unit_list_export = list(set(unit_list_data) & set(unit_list_master))

In [ ]:
#Generate a spearate excel for each unit
for unit in tqdm_notebook(unit_list_export):    
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter('output/RWA_DQ_checks_' + rd + '_' + unit + '_(Timestamp_{}).xlsx'
                            .format(pd.datetime.today().strftime('%Y%m%d_%Hh%M')), engine='xlsxwriter')

    #write overview of checks to first sheet
    df_qc_list_active_nwu.to_excel(writer, sheet_name="QC_list")

    # Write each dataframe to a different worksheet.
    for i in tqdm_notebook(sql_list["ID"]):
        df = d[i]
        df_unit = df[df["unit_code"] == unit]
        if df_unit.empty:
            continue #skip generation of qc sheet in excel if empty
        df_unit.to_excel(writer, sheet_name=i)

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()